In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from k12libs.utils.nb_easy import k12ai_model_summary
import torch
import torchvision

In [12]:
net = torchvision.models.resnet18()
k12ai_model_summary(net, input_size=(32, 3, 224, 224))

(3, 224, 224) 32
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [32, 64, 112, 112]           9,408
       BatchNorm2d-2         [32, 64, 112, 112]             128
              ReLU-3         [32, 64, 112, 112]               0
         MaxPool2d-4           [32, 64, 56, 56]               0
            Conv2d-5           [32, 64, 56, 56]          36,864
       BatchNorm2d-6           [32, 64, 56, 56]             128
              ReLU-7           [32, 64, 56, 56]               0
            Conv2d-8           [32, 64, 56, 56]          36,864
       BatchNorm2d-9           [32, 64, 56, 56]             128
             ReLU-10           [32, 64, 56, 56]               0
       BasicBlock-11           [32, 64, 56, 56]               0
           Conv2d-12           [32, 64, 56, 56]          36,864
      BatchNorm2d-13           [32, 64, 56, 56]             128
             ReLU-14  

In [4]:

device=next(net.parameters()).device
device

device(type='cpu')

In [5]:
import torch

In [6]:
torch.device('cuda:0')

device(type='cuda', index=0)

In [7]:
import torchsummary

In [19]:
from k12libs.utils.nb_easy import k12ai_start_html, W3URL
k12ai_start_html(f'{W3URL}/drawnet.html?jfile=complex', height=780, flask=True)

 * Serving Flask app "k12libs.utils.nb_easy" (lazy loading)
 * Environment: production


   Use a production WSGI server instead.
 * Debug mode: off


In [7]:
c='''
plain_net { name : \"hproject_model_name\" layer {
					 conv {
					  name : \"Conv2d_oaarnrsino\"
					  layer_builder : \"NNTorchLayer\"
					  layer_mode : CONV2D
					  inputs: 'x'
					  outputs : \"Cat_up2wmuwnpr\"
					  layer_params { 
					 	 in_channels : \"3\"  out_channels : \"16\"  kernel_size : \"3\"  stride : \"1\"  padding : \"0\"  dilation : \"1\"  groups : \"1\"  bias : \"True\" 
					 } 
					 }
					 }layer {
					 conv {
					  name : \"Conv2d_4g8syblwxq\"
					  layer_builder : \"NNTorchLayer\"
					  layer_mode : CONV2D
					  inputs: \"Input_g482auoi5c\" 
					  outputs : \"Cat_up2wmuwnpr\"
					  layer_params { 
					 	 in_channels : \"3\"  out_channels : \"16\"  kernel_size : \"3\"  stride : \"1\"  padding : \"0\"  dilation : \"1\"  groups : \"1\"  bias : \"True\" 
					 } 
					 }
					 }layer {
					 basefunc {
					  name : \"Cat_up2wmuwnpr\"
					  layer_builder : \"TorchFuncLayer\"
					  layer_mode : CAT
					  inputs: \"Conv2d_oaarnrsino\" inputs: \"Conv2d_4g8syblwxq\" inputs: \"Conv2d_r1t03pqox2\" inputs: \"Conv2d_cvrd36a04m\" 
					  outputs : \"Flatten_tghk558cme\"
					  layer_params { 
					 	 dim : \"1\" 
					 } 
					 }
					 }layer {
					 vulkan {
					  name : \"Flatten_tghk558cme\"
					  layer_builder : \"NNTorchLayer\"
					  layer_mode : FLATTEN
					  inputs: \"Cat_up2wmuwnpr\" 
					  outputs : \"Linear_3phtruppi\"
					  layer_params { 
					 	 start_dim : \"1\"  end_dim : \"-1\" 
					 } 
					 }
					 }layer {
					 linear {
					  name : \"Linear_3phtruppi\"
					  layer_builder : \"NNTorchLayer\"
					  layer_mode : LINEAR
					  inputs: \"Flatten_tghk558cme\" 
					  outputs : \"Output_pdusvnqf7w\"
					  layer_params { 
					 	 in_features : \"43264\"  out_features : \"10\"  bias : \"False\" 
					 } 
					 }
					 }layer {
					 conv {
					  name : \"Conv2d_r1t03pqox2\"
					  layer_builder : \"NNTorchLayer\"
					  layer_mode : CONV2D
					  inputs: \"Input_g482auoi5c\" 
					  outputs : \"Cat_up2wmuwnpr\"
					  layer_params { 
					 	 in_channels : \"3\"  out_channels : \"16\"  kernel_size : \"3\"  stride : \"1\"  padding : \"0\"  dilation : \"1\"  groups : \"1\"  bias : \"True\" 
					 } 
					 }
					 }layer {
					 conv {
					  name : \"Conv2d_cvrd36a04m\"
					  layer_builder : \"NNTorchLayer\"
					  layer_mode : CONV2D
					  inputs: \"Conv2d_oaarnrsino\" 
					  outputs : \"Cat_up2wmuwnpr\"
					  layer_params { 
					 	 in_channels : \"16\"  out_channels : \"16\"  kernel_size : \"3\"  stride : \"1\"  padding : \"1\"  dilation : \"1\"  groups : \"1\"  bias : \"True\" 
					 } 
					 }
					 } } '''

print(c)


plain_net { name : "hproject_model_name" layer {
					 conv {
					  name : "Conv2d_oaarnrsino"
					  layer_builder : "NNTorchLayer"
					  layer_mode : CONV2D
					  inputs: 'x'
					  outputs : "Cat_up2wmuwnpr"
					  layer_params { 
					 	 in_channels : "3"  out_channels : "16"  kernel_size : "3"  stride : "1"  padding : "0"  dilation : "1"  groups : "1"  bias : "True" 
					 } 
					 }
					 }layer {
					 conv {
					  name : "Conv2d_4g8syblwxq"
					  layer_builder : "NNTorchLayer"
					  layer_mode : CONV2D
					  inputs: "Input_g482auoi5c" 
					  outputs : "Cat_up2wmuwnpr"
					  layer_params { 
					 	 in_channels : "3"  out_channels : "16"  kernel_size : "3"  stride : "1"  padding : "0"  dilation : "1"  groups : "1"  bias : "True" 
					 } 
					 }
					 }layer {
					 basefunc {
					  name : "Cat_up2wmuwnpr"
					  layer_builder : "TorchFuncLayer"
					  layer_mode : CAT
					  inputs: "Conv2d_oaarnrsino" inputs: "Conv2d_4g8syblwxq" inputs: "Conv2d_r1t03pqox2" inputs: "